<b><img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250, style="padding: 10px"> 
<p><p><p><p><p><p>
<b>Interactive Image-Catalog Visualization</b> <br>
Last verified to run on <b>2021-10-08</b> with LSST Science Pipelines release <b>w_2021_33</b> <br>
Contact authors: Keith Bechtol <br>
Target audience: All DP0 delegates. <br>
Container Size: medium or large <br>
Questions welcome at <a href="https://community.lsst.org/c/support/dp0">community.lsst.org/c/support/dp0</a> <br>
Find DP0 documentation and resources at <a href="https://dp0-1.lsst.io">dp0-1.lsst.io</a> <br>

**Credit:** This tutorial builds upon notebooks developed for DP0.1 by Leanne Guy. Please consider acknowledging Leanne Guy and Keith Bechtol in any publications or software releases that make use of this notebook's contents.

### Learning Objectives

This tutorial builds upon ``08a_Interactive_Image_Visualization`` and `08b_Interactive_Catalog_Visualization` as part of a series introducing three open-source Python libraries that enable powerful interactive visualization of images and catalogs. 
 1. [**HoloViews**](http://holoviews.org): Produce high-quality interactive visualizations easily by annotating plots and images rather than using direct calls to a plotting library
 2. [**Bokeh**](https://bokeh.org): A powerful data visualization library that provides interactive tools including brushing and linking between multiple plots. `Holoviews` + `Bokeh`
 3. [**Datashader**](https://datashader.org): Accurately render very large datasets quickly and flexibly.
  
These packages are part of the [Holoviz](https://holoviz.org/) ecosystem of tools intended for visualization in a web browser and can be used to create quite sophisticated dashboard-like interactive displays and widgets. The goal of this tutorial is to provide an introduction and starting point from which to create more advanced, custom interactive visualizations. Holoviz has a [vibrant and active community](https://discourse.holoviz.org/) where you can ask questions and discuss vizualizations with a global community. 

This tutorial focuses on interaction between image and catalog displays.

### Logistics
This notebook is intended to be runnable on `data.lsst.cloud`. Note that occasionally the notebook may seem to stall, or the interactive features may seem disabled. If this happens, usually a restart of the kernel fixes the issue. You might also need to log out of the RSP and start a "large" instance of the JupyterLab environment. In some examples shown in this notebook, the order in which the cells are run is important for understanding the interactive features, so you may want to re-run the set of cells in a given section if you encounter unexpected behavior. Note that some of the examples require manual selection of points on a graph to run correctly.

### Setup

In [ ]:
# General python imports
import numpy as np

# Astropy
from astropy.visualization import  ZScaleInterval, AsinhStretch

# LSST imports
from lsst.daf.butler import Butler
import lsst.geom as geom

# Bokeh and Holoviews for visualization
import bokeh
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource

import holoviews as hv
from holoviews import streams
from holoviews.operation.datashader import rasterize

import panel as pn

# Set the holoviews plotting library to be bokeh
# You will see the holoviews + bokeh icons displayed when the library is loaded successfully
hv.extension('bokeh')

# Display bokeh plots inline in the notebook
output_notebook()

In [ ]:
# What versions of bokeh and holoviews nd datashader are we working with?
# This is important when referring to online documentation as
# APIs can change between versions.
print("Bokeh version: " + bokeh.__version__)
print("Holoviews version: " + hv.__version__)

In [ ]:
# What version of the LSST Science Pipelnes are we using?
! echo $IMAGE_DESCRIPTION
! eups list -s | grep lsst_distrib

### 1. Data Preparation

Follow the data preparation steps from `08a_Interactive_Image_Visualization` to load 
1. a image for a single detector and the associated source catalog
2. a coadd image for a single patch and the associated object catalog.

In [ ]:
# Instantiate the Butler initializing it with the repository name and the DP0.1 collection identifier
#from lsst.daf.butler import Butler
repo = 's3://butler-us-central1-dp01'
collection = '2.2i/runs/DP0.1'
butler = Butler(repo, collections=collection)

In [ ]:
# Define a calibrated exposure and retrieve it via the Butler
calexpId = {'visit': 192350, 'detector': 175, 'band': 'i'}
calexp = butler.get('calexp', **calexpId)
assert calexp is not None
# Source table for this exposure
calexpSrc = butler.get('src', **calexpId)

In [ ]:
# Define a deep coadded image and retrieve it via the Butler
coaddId = {'tract': 4226, 'patch': 17, 'band': 'r'}
coadd = butler.get('deepCoadd', **coaddId)
assert coadd is not None
# Source table for this coadd
coaddSrc = butler.get('deepCoadd_forced_src', coaddId)

### 2. Interactively Select Objects Overlaid on an Image with an Adjustable Colorbar Range

This example demonstrates some possibilities for selecting catalog objects based on an image. We create an interactive dashboard with [**panel**](https://panel.holoviz.org/), a package in the [**HoloViz**](https://holoviz.org/) ecosystem designed to create custom interactive web apps and dashboards. In this case, we add sliders to interactively adjust the contrast of the image. More inspiration for interactive dashboards can found in the [**HoloViews User Guide**](https://holoviews.org/user_guide/Dashboards.html).

In [ ]:
# Apply a asinh/zscale mapping to the data 
transform = AsinhStretch() + ZScaleInterval()
scaledImage = transform(calexp.image.array)

scaledImage = np.flipud(scaledImage)
bounds_img = (0, 0, calexp.getDimensions()[0], calexp.getDimensions()[1])

s = calexpSrc.getColumnView()
coords = s.getX(), s.getY()
detections = hv.Points(coords).opts(fill_color=None, size=9, color="darkorange",
                                    tools=['hover', 'box_select', 'lasso_select'])

def makeImage(zmin=0., zmax=1.0):
    """Update image colorbar range."""
    image = hv.Image(scaledImage, 
                     bounds=bounds_img, 
                     vdims=hv.Dimension('z', range=(zmin, zmax))).opts(
        cmap = "Greys", colorbar=True, height=600, width=700)
    return image

zmin = pn.widgets.FloatSlider(name='zmin', value=0.5, start=0., end=1., step=0.01)
zmax = pn.widgets.FloatSlider(name='zmax', value=1., start=0., end=1., step=0.01)

dmap = hv.DynamicMap(pn.bind(makeImage, zmin=zmin, zmax=zmax))

selection = streams.Selection1D(source=detections)

app = pn.Row(pn.WidgetBox('Colorbar Range', zmin, zmax), rasterize(dmap) * detections)
app

> STOP - Select some data points from the plot above using the lasso or box select tool. Notice that the selection updates.

In [ ]:
selection

### 3. Interactively Select Objects from a Scatter Plot and Create Postage Stamp Cutouts

In this example, we demonstrate the possibility to trigger more advanced analysis within an interactive display. In one panel we create a scatter plot based on catalog data, and in a second panel we generate a postage stamp cutout image for an object selected on the scatter plot. Each time we select a different object, we retrieve the cutout image.

We directly borrow the postage stamp image cutout function from the `03_Image_Display_and_Manipulation` tutorial.

In [ ]:
def cutout_coadd(butler, ra, dec, band='r', datasetType='deepCoadd',
                 skymap=None, cutoutSideLength=51, **kwargs):
    """
    Produce a cutout from a coadd at the given ra, dec position.

    Adapted from DC2 tutorial notebook by Michael Wood-Vasey.

    Parameters
    ----------
    butler: lsst.daf.persistence.Butler
        Servant providing access to a data repository
    ra: float
        Right ascension of the center of the cutout, in degrees
    dec: float
        Declination of the center of the cutout, in degrees
    band: string
        Filter of the image to load
    datasetType: string ['deepCoadd']
        Which type of coadd to load.  Doesn't support 'calexp'
    skymap: lsst.afw.skyMap.SkyMap [optional]
        Pass in to avoid the Butler read.  Useful if you have lots of them.
    cutoutSideLength: float [optional]
        Size of the cutout region in pixels.

    Returns
    -------
    MaskedImage
    """
    radec = geom.SpherePoint(ra, dec, geom.degrees)
    cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)

    if skymap is None:
        skymap = butler.get("skyMap")

    # Look up the tract, patch for the RA, Dec
    tractInfo = skymap.findTract(radec)
    patchInfo = tractInfo.findPatch(radec)
    xy = geom.PointI(tractInfo.getWcs().skyToPixel(radec))
    bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
    patch = tractInfo.getSequentialPatchIndex(patchInfo)

    coaddId = {'tract': tractInfo.getId(), 'patch': patch, 'band': band}
    parameters = {'bbox': bbox}

    cutout_image = butler.get(datasetType, parameters=parameters,
                              immediate=True, dataId=coaddId)

    return cutout_image

Now we are ready to create the interactive display. Try selecting different objects in the scatter plot and see the cutout image update.

In [ ]:
source = ColumnDataSource(data=dict(ra=np.degrees(coaddSrc['coord_ra']),
                                    dec=np.degrees(coaddSrc['coord_dec']),
                                    flux=coaddSrc['modelfit_CModel_instFlux'],
                                    fluxErr=coaddSrc['modelfit_CModel_instFluxErr'],
                                    objectId=coaddSrc['id']))

# Scatter plot of detected objects
# We can specify the axis to plot as well as additional information
detections = hv.Points(kdims=['flux', 'fluxErr'], 
                       vdims=['ra', 'dec', 'objectId'], 
                       data=source.data)
detections.opts(
    tools=['tap'], 
    height=600, width=600, logx=True, logy=True, xlim=(1, 1.e5),
    fill_color='darkorange', size=9, color="darkorange",
    nonselection_fill_color='none', nonselection_line_color='black')

# Selection from the scatter plot
selection = streams.Selection1D(source=detections)

def makeImageCutout(index):
    """Retrieve cutout image for a selected object."""
    if len(index) == 0:
        index = [0]
    
    ra = detections.data['ra'][index].values[0]
    dec = detections.data['dec'][index].values[0]
    objectId = detections.data['objectId'][index].values[0]
    
    cutout = cutout_coadd(butler, ra, dec)
    bounds_img = (0, 0, cutout.getDimensions()[0], cutout.getDimensions()[1])
    
    image = hv.Image(cutout.image.array, bounds=bounds_img).opts(
        cmap = "Greys", colorbar=True, height=600, width=700)
    
    image = image.relabel("(RA, Dec) = (%.6f, %.6f) ; objectId = %i"%(ra, dec, objectId))
    return image

dmap = hv.DynamicMap(makeImageCutout, streams=[selection])

# The axiswise=True allows the axes of the two panels to be independent of each other
app = pn.Row(detections.opts(axiswise=True), 
             rasterize(dmap).opts(axiswise=True))
app